In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

import torch
import torch.optim as optim

from scipy.spatial import ConvexHull
from matplotlib.patches import Polygon
import sys,os
sys.path.append(os.path.realpath('./src/'))
import pandas as pd
from materialEncoder import MaterialEncoder
import time
import itertools
%matplotlib qt

In [2]:
def to_np(x):
  return x.detach().cpu().numpy()

def to_torch(x):
  return torch.tensor(x).float()

In [3]:
def preprocessData():
  df = pd.read_excel('./data/solidworksMaterialDatabaseCost.xlsx')# #  #subsetSolidworksDatabase
  dataIdentifier = {'name': df[df.columns[0]], 'className':df[df.columns[1]], 'classID':df[df.columns[2]]} # name of the material and type
  trainInfo = np.log10(df[df.columns[3:]].to_numpy())
  dataScaleMax = torch.tensor(np.max(trainInfo, axis = 0))
  dataScaleMin = torch.tensor(np.min(trainInfo, axis = 0))
  normalizedData = (torch.tensor(trainInfo) - dataScaleMin)/(dataScaleMax - dataScaleMin)
  trainingData = torch.tensor(normalizedData).float()
  dataInfo = {'youngsModulus':{'idx':0,'scaleMin':dataScaleMin[0], 'scaleMax':dataScaleMax[0]},\
              'costPerKg':{'idx':1,'scaleMin':dataScaleMin[1], 'scaleMax':dataScaleMax[1]},\
              'massDensity':{'idx':2,'scaleMin':dataScaleMin[2], 'scaleMax':dataScaleMax[2]},\
              'yieldStrength':{'idx':3,'scaleMin':dataScaleMin[3], 'scaleMax':dataScaleMax[3]}}
  return trainingData, dataInfo, dataIdentifier, trainInfo
trainingData, dataInfo, dataIdentifier, trainInfo = preprocessData()
numMaterialsInTrainingData, numFeatures = trainingData.shape

<ipython-input-3-f0ec4a539955>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainingData = torch.tensor(normalizedData).float()


In [4]:
latentDim, hiddenDim = 2, 250
numEpochs = 50000
klFactor = 5e-5
learningRate = 2e-3
savedNet = './data/vaeNet.nt'
vaeSettings = {'encoder':{'inputDim':numFeatures, 'hiddenDim':hiddenDim,\
                                          'latentDim':latentDim},\
               'decoder':{'latentDim':latentDim, 'hiddenDim':hiddenDim,\
                                          'outputDim':numFeatures}}

materialEncoder = MaterialEncoder(trainingData, dataInfo, dataIdentifier, vaeSettings)

start = time.perf_counter()
convgHistory = materialEncoder.trainAutoencoder(numEpochs, klFactor, savedNet, learningRate)
print('training time : {:.2F} '.format(time.perf_counter() - start))


Iter 0 reconLoss 3.09E+01 klLoss 1.25E-02 loss 3.10E+01
Iter 500 reconLoss 2.80E-01 klLoss 6.00E-02 loss 3.40E-01
Iter 1000 reconLoss 1.78E-01 klLoss 5.75E-02 loss 2.35E-01
Iter 1500 reconLoss 1.37E-01 klLoss 5.45E-02 loss 1.91E-01
Iter 2000 reconLoss 1.02E-01 klLoss 5.28E-02 loss 1.55E-01
Iter 2500 reconLoss 7.84E-02 klLoss 5.14E-02 loss 1.30E-01
Iter 3000 reconLoss 6.64E-02 klLoss 5.04E-02 loss 1.17E-01
Iter 3500 reconLoss 5.57E-02 klLoss 4.95E-02 loss 1.05E-01
Iter 4000 reconLoss 5.21E-02 klLoss 4.81E-02 loss 1.00E-01
Iter 4500 reconLoss 4.60E-02 klLoss 4.72E-02 loss 9.31E-02
Iter 5000 reconLoss 3.54E-02 klLoss 4.63E-02 loss 8.17E-02
Iter 5500 reconLoss 3.26E-02 klLoss 4.55E-02 loss 7.81E-02
Iter 6000 reconLoss 2.83E-02 klLoss 4.48E-02 loss 7.31E-02
Iter 6500 reconLoss 2.61E-02 klLoss 4.45E-02 loss 7.06E-02
Iter 7000 reconLoss 2.42E-02 klLoss 4.41E-02 loss 6.82E-02
Iter 7500 reconLoss 2.17E-02 klLoss 4.29E-02 loss 6.46E-02
Iter 8000 reconLoss 2.01E-02 klLoss 4.29E-02 loss 6.30E-02
I

In [5]:
from examples import getExample
from trussFE import TrussFE

exampleName, nodeXY, connectivity, bc = getExample(4)
bc['forces'] = {'nodes':np.array([2]), 'fx':1.E3*torch.tensor([0.]), 'fy':1.E2*torch.tensor([125.])}
truss = TrussFE(nodeXY, connectivity, bc)


A = 2e-3*torch.ones((connectivity.shape[0]))
u, dispX, dispY, nodalDeformation, internalForce = truss.solveFE(E = torch.tensor([2e11]).float(), A =A)
truss.plot(f'J {truss.computeCompliance(u):.2E} , volume {truss.getVolume(A):.2E}', plotDeformed = False)

In [6]:
# lossMethod = {'type':'penalty', 'alpha0':0.05, 'delAlpha':0.15}
lossMethod = {'type':'logBarrier', 't0':3, 'mu':1.001}
optParams = {'minEpochs':150, 'maxEpochs':900, 'lossMethod':lossMethod,'lr':0.001,'gradclip':{'isOn':True, 'thresh':2e-1}}

In [7]:
plt.close('all')

import torch
import torch.optim as optim
import numpy as np
import itertools
import matplotlib.pyplot as plt
from networks import TopologyNetwork, MaterialNetwork
from utilFuncs import to_torch
import torch.nn as nn
lkyReLU = nn.LeakyReLU(0.00)

class TrussOptimization:
  def __init__(self, truss, materialEncoder):
    self.truss = truss
    self.materialEncoder = materialEncoder
    self.verbose = True
  #--------------------------#
  def computeMetrics(self, area, properties):
    metrics = {}
    u, _, _, _, internalForce = self.truss.solveFE(properties['youngsModulus'], area)
    metrics['compliance'] = self.truss.computeCompliance(u)
    metrics['volume'] = self.truss.getVolume(area)
    metrics['mass'] = properties['massDensity']*metrics['volume']
    metrics['cost'] = metrics['mass']*properties['costPerKg']
    return metrics, u, internalForce
  #--------------------------#
  def computeConstraints(self, constraints, metrics, area, internalForce, properties):
    #~~~~~~~~~~~#
    def pnormMax(x, p = 6.):
      return torch.pow(torch.sum(x**p), 1./p)
    #~~~~~~~~~~~#
    if(constraints['massCons']['isOn']):
      constraints['massCons']['value'] = (metrics['mass']/constraints['massCons']['maxMass']) - 1.
    #~~~~~~~~~~~#
    if(constraints['costCons']['isOn']):
      constraints['costCons']['value'] = (metrics['cost']/constraints['costCons']['maxCost']) - 1.
    #~~~~~~~~~~~#
    if(constraints['volumeCons']['isOn']):
      constraints['volumeCons']['value'] = \
        (metrics['volumeCons']/constraints['volumeCons']['desiredVolume']) - 1.
    #~~~~~~~~~~~#
    if(constraints['tensionCons']['isOn']):
      tensileForce = 0.001+torch.relu(internalForce)
      tensileStress = tensileForce/area
      maxTensileStressExperienced = pnormMax(tensileStress)*constraints['tensionCons']['FOS']
      constraints['tensionCons']['value'] = (maxTensileStressExperienced/properties['yieldStrength']) - 1.
    #~~~~~~~~~~~#
    if(constraints['compressionCons']['isOn']):
      compressiveForce = 0.001+torch.relu(-internalForce)
      k = 4. # used in geom factor... see buckling theory
      geomFactor = k*(self.truss.barLength/(np.pi*area))**2
      compressiveStress = compressiveForce*geomFactor
      maxCompressiveStressExperienced = pnormMax(compressiveStress)*constraints['compressionCons']['FOS']
      constraints['compressionCons']['value'] = \
        (maxCompressiveStressExperienced/properties['youngsModulus']) - 1.
    return constraints     
  #--------------------------#
  def optimizeDesign(self, optParams, areaOptimization, materialOptimization, constraints):
    #~~~~~~~~~~~#
    def computeConstraintTerm(consVal):
      if(optParams['lossMethod']['type'] == 'penalty'):
        alpha = min(100.,optParams['lossMethod']['alpha0'] + \
                epoch*optParams['lossMethod']['delAlpha']) # penalty method
        consTerm = alpha*lkyReLU(consVal)#**2
      if(optParams['lossMethod']['type'] == 'logBarrier'):
        t = optParams['lossMethod']['t0']* \
                          optParams['lossMethod']['mu']**epoch;
        if(consVal < (-1/t**2)):
          consTerm = -torch.log(-consVal)/t
        else:
          consTerm = t*consVal - np.log(1/t**2)/t + 1./t
      return consTerm
    #~~~~~~~~~~~#
    def rescale(val, maxVal, minVal):
      return 10.**(minVal + val*(maxVal - minVal))
    #~~~~~~~~~~~#
    designVariables = []
    if(areaOptimization['isOn']):
      topNet = TopologyNetwork(areaOptimization['netSettings'])
      xyBarCenter = to_torch(self.truss.barCenter)
      designVariables = itertools.chain(designVariables,topNet.parameters())
    if(materialOptimization['isOn']):
      matNet = MaterialNetwork(materialOptimization['netSettings'])
      matInput = torch.tensor([1.], requires_grad = True).float().view(-1,1)
      designVariables = itertools.chain(designVariables,matNet.parameters())
    #~~~~~~~~~~~#
    metrics = {'compliance':0., 'volume':0., 'mass':0., 'cost':0.}
    convergenceHistory = {} # keep track of metrics and constraints
    for k in metrics:
      convergenceHistory[k] = []
    for c in constraints:
      convergenceHistory[c] = []
    obj0 = 1.

    optimizer = optim.Adagrad(designVariables, lr=0.02)
    #~~~~~~~~~~~#
    print("iter \t J \t vol \t mass \t cost")
    for epoch in range(optParams['maxEpochs']):
      #~~~~~~~~~~~#
      def getMaterialProperties(decoded):
        properties = {}
        keys = ['youngsModulus', 'costPerKg', 'massDensity', 'yieldStrength']
        dataInfo = self.materialEncoder.dataInfo # too lazy to write :D
        for k in keys:
          properties[k] = rescale(decoded[:,dataInfo[k]['idx']],\
                          dataInfo[k]['scaleMax'],\
                          dataInfo[k]['scaleMin'])
        return properties
      #~~~~~~~~~~~#
      if(areaOptimization['isOn']):
        nnOut = topNet(xyBarCenter)
        self.area = areaOptimization['bounds']['min'] + \
          (areaOptimization['bounds']['max'] - \
           areaOptimization['bounds']['min'])*nnOut
      else:
        self.area = areaOptimization['area']
      #~~~~~~~~~~~#
      if(materialOptimization['isOn']):
        self.optimalZ = matNet(matInput)
        decoded = self.materialEncoder.vaeNet.decoder(self.optimalZ) 
        properties = getMaterialProperties(decoded)
      else:
        properties = materialOptimization['properties']
      #~~~~~~~~~~~#
      metrics, u, internalForce = self.computeMetrics(self.area, properties)
      #~~~~~~~~~~~#
      constraints = self.computeConstraints(constraints, metrics, self.area, internalForce, properties)
      #~~~~~~~~~~~#
      loss = (metrics['compliance']/obj0)
      for c in constraints:
        if(constraints[c]['isOn']):
          loss = loss + computeConstraintTerm(constraints[c]['value'])
          convergenceHistory[c].append(constraints[c]['value'].item())
      #~~~~~~~~~~~#
      if(epoch == 0):
        obj0 = metrics['compliance'].item()
      #~~~~~~~~~~~#
      loss.backward(retain_graph=True)
      if(areaOptimization['isOn'] and optParams['gradclip']['isOn']):
        torch.nn.utils.clip_grad_norm_(topNet.parameters(),optParams['gradclip']['thresh'])
#       if(materialOptimization['isOn'] and optParams['gradclip']['isOn']):
#         torch.nn.utils.clip_grad_norm_(matNet.parameters(),optParams['gradclip']['thresh'])
      optimizer.step()
      #~~~~~~~~~~~#
      titleStr = f" {epoch} \t {metrics['compliance'].item():.2E} \t \
         {metrics['volume'].item():.2E} \t {metrics['mass'].item():.2E} \t \
         {metrics['cost'].item():.2E}"
      if(self.verbose and epoch%10 == 0):
        print(titleStr)
      #~~~~~~~~~~~#
      for c in constraints:
        if(constraints[c]['isOn']):
          convergenceHistory[c].append(constraints[c]['value'].item())
      for k in metrics:
        convergenceHistory[k].append(metrics[k].item())
      #~~~~~~~~~~~#
      if(epoch%50 ==0 and self.verbose):
        self.truss.plot(f"iter {epoch}, J {metrics['compliance'].item():.2E}", False)
        plt.pause(0.001)
    print(titleStr)
    print('-------\n--constraints------\n')
    for c in constraints:
      if(constraints[c]['isOn']):
        print(f" {c} : {constraints[c]['value'].item():.2E}")
    print('---------------\n')
    return convergenceHistory

In [8]:
# from trussOptimization import TrussOptimization
topNetSettings = {'inputDim':2, 'numLayers':2, 'numNeuronsPerLyr':20, 'outputDim':1}
matNetSettings = {'inputDim':1, 'numLayers':2,\
                  'numNeuronsPerLyr':20, 'outputDim':latentDim, 'zMin':-1., 'zRange':3.}

constraints = {'volumeCons': {'isOn':False, 'desiredVolume':2e-6},\
               'massCons': {'isOn':True, 'maxMass':4E1}, \
               'costCons': {'isOn':False, 'maxCost':5e1},\
               'tensionCons': {'isOn':True, 'FOS':4.},\
               'compressionCons': {'isOn':True, 'FOS':4.}}

areaOptimization = {'isOn':True, 'bounds':{'min':1e-9, 'max':1e-2}, 'netSettings':topNetSettings}
# areaOptimization = {'isOn':False, 'area':2e-3*torch.ones((connectivity.shape[0]), requires_grad = True)}

materialOptimization = {'isOn':True, 'netSettings':matNetSettings}
# materialOptimization = {'isOn':False, 'properties':{'youngsModulus':torch.tensor([1.e9]),\
#                                                     'massDensity':torch.tensor([1.e3]),\
#                                                     'yieldStrength':torch.tensor([1.e7]), \
#                                                     'costPerKg':torch.tensor([4.])}}

trussopt = TrussOptimization(truss, materialEncoder)

plt.close('all')
convergenceHistory = trussopt.optimizeDesign(optParams, areaOptimization, materialOptimization, constraints)

iter 	 J 	 vol 	 mass 	 cost
 0 	 4.42E+00 	          3.82E-02 	 1.03E+02 	          2.24E+02
 10 	 7.34E+00 	          1.92E-02 	 5.18E+01 	          1.13E+02
 20 	 1.01E+01 	          1.29E-02 	 3.47E+01 	          7.64E+01
 30 	 1.00E+01 	          1.29E-02 	 3.49E+01 	          7.63E+01
 40 	 9.86E+00 	          1.29E-02 	 3.48E+01 	          7.77E+01
 50 	 9.81E+00 	          1.29E-02 	 3.47E+01 	          7.74E+01
 60 	 9.60E+00 	          1.30E-02 	 3.61E+01 	          8.63E+01
 70 	 9.76E+00 	          1.29E-02 	 3.57E+01 	          8.45E+01
 80 	 9.78E+00 	          1.30E-02 	 3.49E+01 	          8.13E+01
 90 	 9.62E+00 	          1.33E-02 	 3.57E+01 	          8.19E+01
 100 	 9.54E+00 	          1.34E-02 	 3.59E+01 	          8.59E+01
 110 	 9.48E+00 	          1.34E-02 	 3.60E+01 	          9.00E+01
 120 	 9.48E+00 	          1.29E-02 	 3.49E+01 	          9.23E+01
 130 	 9.47E+00 	          1.28E-02 	 3.50E+01 	          9.18E+01
 140 	 9.42E+00 	          1.30E-02 	 3.57E+

In [11]:
print(trussopt.area)

tensor([0.0029, 0.0024, 0.0023, 0.0028, 0.0009, 0.0011],
       grad_fn=<AddBackward0>)


In [12]:
print(trussopt.optimalZ)

tensor([[-0.1253, -0.2210]], grad_fn=<AddBackward0>)


In [ ]:
from utilFuncs import plotConvergence
plotConvergence(convergenceHistory)

In [9]:
matToUse = trussopt.materialEncoder.getClosestMaterialFromZ(trussopt.optimalZ, numClosest =3)
print(matToUse['material'][0])


ltnt1, ltnt2 = 0, 1
fig, ax = materialEncoder.plotLatent(ltnt1 = ltnt1, ltnt2 = ltnt2, plotHull = False, annotateHead = False,\
                                    saveFileName = './figures/optimalLatent.pdf')
zOpt_np = to_np(trussopt.optimalZ)
for i in range(zOpt_np.shape[0]):
  ax.annotate('OPTIMAL MATERIAL {:d}'.format(i+1), (zOpt_np[i,ltnt1], zOpt_np[i,ltnt2]))
  ax.scatter(zOpt_np[i,ltnt1], zOpt_np[i,ltnt2], marker='*',s=400)

closest material 0 : 2018 , confidence 94.15
closest material 1 : AA380.0-F die , confidence 90.92
closest material 2 : 2014-T4 , confidence 89.83
2018


### Re-optimize with nearest material

In [10]:
from utilFuncs import to_torch
ind = np.where(dataIdentifier['name'] == matToUse['material'][0])
matData = 10.**trainInfo[ind,:].reshape(-1)
properties = {'name': matToUse['material'][0], \
               'youngsModulus':to_torch(matData[0]), \
               'costPerKg':to_torch(matData[1]), \
               'massDensity':to_torch(matData[2]), \
               'yieldStrength': to_torch(matData[3])}

materialOptimization = {'isOn':False, 'properties':properties}
convergenceHistory = trussopt.optimizeDesign(optParams, areaOptimization, materialOptimization, constraints)

iter 	 J 	 vol 	 mass 	 cost
 0 	 4.15E+00 	          3.82E-02 	 1.07E+02 	          2.90E+02
 10 	 6.95E+00 	          1.91E-02 	 5.35E+01 	          1.45E+02
 20 	 8.99E+00 	          1.36E-02 	 3.81E+01 	          1.03E+02
 30 	 9.18E+00 	          1.32E-02 	 3.68E+01 	          1.00E+02
 40 	 9.25E+00 	          1.31E-02 	 3.66E+01 	          9.92E+01
 50 	 9.29E+00 	          1.30E-02 	 3.64E+01 	          9.89E+01
 60 	 9.32E+00 	          1.30E-02 	 3.63E+01 	          9.86E+01
 70 	 9.31E+00 	          1.30E-02 	 3.63E+01 	          9.86E+01
 80 	 9.34E+00 	          1.29E-02 	 3.62E+01 	          9.84E+01
 90 	 9.36E+00 	          1.29E-02 	 3.62E+01 	          9.81E+01
 100 	 9.37E+00 	          1.29E-02 	 3.61E+01 	          9.81E+01
 110 	 9.39E+00 	          1.29E-02 	 3.61E+01 	          9.79E+01
 120 	 9.39E+00 	          1.29E-02 	 3.61E+01 	          9.78E+01
 130 	 9.39E+00 	          1.29E-02 	 3.61E+01 	          9.79E+01
 140 	 9.41E+00 	          1.29E-02 	 3.60E+

### Bruteforce

In [ ]:
trussopt.verbose = False
z = trussopt.materialEncoder.vaeNet.encoder.z.to('cpu').detach().numpy()
print('material \t z0 \t z1 \t J \t volume \t mass \t cost \t massCons \t tensionCons \t compressionCons ')
bruteProp = {'material':[], 'J':[], 'cost':[], 'mass':[], 'z0':[], 'z1':[], 'tensionCons':[], 'compressionCons':[]}
for ind in range(trainInfo.shape[0]):
  print(dataIdentifier['name'][ind], end = '\t')
  print(f"{z[ind,0]:.2E} \t {z[ind,1]:.2E}", end = '\t')
  matData = 10.**trainInfo[ind,:].reshape(-1)
  properties = {'name': matToUse['material'][0], \
                 'youngsModulus':to_torch(matData[0]), \
                 'costPerKg':to_torch(matData[1]), \
                 'massDensity':to_torch(matData[2]), \
                 'yieldStrength': to_torch(matData[3])}

  materialOptimization = {'isOn':False, 'properties':properties}
#   convergenceHistory = trussopt.optimizeDesign(optParams, areaOptimization, materialOptimization, constraints)
  metrics, u, internalForce = trussopt.computeMetrics(areaOptimization['area'],  properties)
  constraints = trussopt.computeConstraints(constraints, metrics, areaOptimization['area'], internalForce)
  for k in metrics:
    print(f' {metrics[k].item():.2E}', end = '\t')
  for c in constraints:
    if(constraints[c]['isOn']):
      print(f"{constraints[c]['value'].item():.2E}", end = '\t')
  print('\n')
  bruteProp['material'].append(dataIdentifier['name'][ind])
  bruteProp['J'].append(metrics['compliance'].item())
  bruteProp['cost'].append(metrics['cost'].item())
  bruteProp['mass'].append(metrics['mass'].item())
  bruteProp['z0'].append(z[ind,0].item())
  bruteProp['z1'].append(z[ind,1].item())
  bruteProp['tensionCons'].append(constraints['tensionCons']['value'].item())
  bruteProp['compressionCons'].append(constraints['compressionCons']['value'].item())

In [ ]:
def plotLatent(prop):
    clrs = ['purple', 'green', 'orange', 'pink', 'yellow', 'black', 'violet', 'cyan', 'red', 'blue']
    colorcol = dataIdentifier['classID']
    ptLabel = dataIdentifier['name']
    autoencoder = trussopt.materialEncoder.vaeNet
    z = autoencoder.encoder.z.to('cpu').detach().numpy()
    fig, ax = plt.subplots()
    a = ax.scatter(z[:, 0], z[:, 1], c = prop, s = 16, cmap = 'rainbow')
    plt.colorbar(a)
    for i in range(np.max(colorcol)+1): 
      zMat = np.vstack((z[colorcol == i,0], z[colorcol == i,1])).T #pts only in a certain class
      if(i == np.max(colorcol)): #removed for last class TEST
        break # END TEST
      hull = ConvexHull(zMat)
      cent = np.mean(zMat, 0)
      pts = []
      for pt in zMat[hull.simplices]:
          pts.append(pt[0].tolist())
          pts.append(pt[1].tolist())

      pts.sort(key=lambda p: np.arctan2(p[1] - cent[1],
                                      p[0] - cent[0]))
      pts = pts[0::2]  # Deleting duplicates
      pts.insert(len(pts), pts[0])
      poly = Polygon(1.1*(np.array(pts) - cent) + cent,
                     facecolor= 'none', alpha=1, edgecolor = 'black') #'black'
      poly.set_capstyle('round')
      plt.gca().add_patch(poly)
      ax.annotate(dataIdentifier['className'][i], (cent[0], cent[1]), size = 12)

    plt.xlabel('$z_0$', size = 18)
    plt.ylabel('$z_1$', size = 18)
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    return fig, ax
  
fig, ax = plotLatent(np.log10(bruteProp['J']))
ax.set_title('compliance')
plt.annotate('Opt. mat.', (zOpt_np[0,ltnt1], zOpt_np[0,ltnt2]))
plt.scatter(zOpt_np[0,0], zOpt_np[0,1], marker='*',s=400)

fig, ax = plotLatent(np.log10(bruteProp['mass']))
ax.set_title('mass')
plt.annotate('Opt. mat.', (zOpt_np[0,ltnt1], zOpt_np[0,ltnt2]))
plt.scatter(zOpt_np[0,0], zOpt_np[0,1], marker='*',s=400)

fig, ax = plotLatent(bruteProp['tensionCons'])
ax.set_title('tension constraint')
plt.annotate('Opt. mat.', (zOpt_np[0,ltnt1], zOpt_np[0,ltnt2]))
plt.scatter(zOpt_np[0,0], zOpt_np[0,1], marker='*',s=400)

fig, ax = plotLatent(np.log10(bruteProp['cost']))
ax.set_title('cost')
plt.annotate('Opt. mat.', (zOpt_np[0,ltnt1], zOpt_np[0,ltnt2]))
plt.scatter(zOpt_np[0,0], zOpt_np[0,1], marker='*',s=400)

fig, ax = plotLatent(bruteProp['compressionCons'])
ax.set_title('compression constraint')
plt.annotate('Opt. mat.', (zOpt_np[0,ltnt1], zOpt_np[0,ltnt2]))
plt.scatter(zOpt_np[0,0], zOpt_np[0,1], marker='*',s=400)